<a href="https://colab.research.google.com/github/shaancodes/zzz/blob/gh-pages/async_yf_plotly_live.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install asyncio yfinance plotly nsetools

In [ ]:
import asyncio
import yfinance as yf
import plotly.express as px
import pandas as pd
import plotly.subplots as sp
import plotly.graph_objects as go
import nsetools 
from IPython.display import clear_output

async def main():
    while True:
        nses = nsetools.Nse()
        top_gainers = nses.get_top_gainers()
        top_gainers =pd.DataFrame(top_gainers)
        top_gainers = top_gainers.head(5)

        top_losers = nses.get_top_losers()
        top_losers =pd.DataFrame(top_losers)
        top_losers = top_losers.head(5)

        def add_ns(symbol):
          return symbol + ".NS"
        top_gainers.iloc[:, 0] = top_gainers.iloc[:, 0].apply(add_ns)
        top_losers.iloc[:, 0] = top_losers.iloc[:, 0].apply(add_ns)

        fig1 = sp.make_subplots(rows=1, cols=1)

        # Add a trace for each top gaining stock
        for symbol in top_gainers['symbol']:
            data = yf.download(tickers=symbol, period='1d', interval='5m')
            data['pct_change'] = (data['Close'] - data.iloc[0]['Open']) / data.iloc[0]['Open'] * 100
            fig1.add_trace(go.Scatter(x=data.index, y=data['pct_change'], name=symbol))

        # Create figure for top losers
        fig2 = sp.make_subplots(rows=1, cols=1)

        # Add a trace for each top losing stock
        for symbol in top_losers['symbol']:
            data = yf.download(tickers=symbol, period='1d', interval='1m')
            data['pct_change'] = (data['Close'] - data.iloc[0]['Open']) / data.iloc[0]['Open'] * 100
            fig2.add_trace(go.Scatter(x=data.index, y=data['pct_change'], name=symbol))

        # Display both figures
        clear_output(wait=True)         
        fig1.show()
        fig2.show()
        await asyncio.sleep(60)

await main()